In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pickle
pickle_file = open('/content/drive/MyDrive/dataframe', "rb")       
df_covid = pickle.load(pickle_file)
pickle_file.close()

In [5]:
df_covid

,paper_id,abstract,body_text,abstract_word_count,body_word_count,body_unique_words
0,00006903b396d50cc0037fed39916d57d50ee801,urban green space regarded contributing citize...,advantages urban green space public health urb...,191,4995,1541
1,0000fcce604204b1b9d876dc073eb529eb5ce305,contribución de los autores rocíof menéndez co...,se calcula que en españa existen cerca de 367 ...,401,4965,1660
2,000122a9a774ec76fa35ec0c0f6734e7e8d0c541,introduction objectives covid 19 outbreak uncl...,introduction objectives covid 19 outbreak uncl...,286,3482,1296
3,0001418189999fea7f7cbe3e82703d71c85a6fe5,feline infectious peritonitis virus fipv posit...,feline infectious peritonitis fip fatal chroni...,164,2049,763
4,00031cc15aafa30b187ed2cd3790e970e5046895,background sars cov 2 infection because renal ...,increased serum creatinine variable degrees pr...,98,4190,1493
...,...,...,...,...,...,...
117686,fffc2cc19b86ddf20cfda2934854909cac98667e,abbreviations arbs angiotensin receptor blocke...,following outbreak novel corona virus infectio...,92,1374,671
117687,ffff2fb21d4880b63a7cf3849cecc45bc596f4f0,prospectively compared efficacy pcr detection ...,rapid detection novel coronavirus sars cov 2 c...,111,1476,552
117688,ffff4330337614d8a69bf518f7b60652b37b14cf,protect human life science public health need ...,started innocently enough with lowgrade fever ...,52,971,569
117689,ffff73d17bc392ee68f3f16ef37d25579cb99322,studies activity rhythms pivotal management in...,invasive alien species represent main causes c...,187,3721,1323


In [6]:
df_covid=df_covid[:10000]

In [7]:
!pip install semanticscholar

In [8]:
!pip install sentence_transformers

     |████████████████████████████████| 81kB 6.1MB/s 
     |████████████████████████████████| 2.0MB 11.6MB/s 
     |████████████████████████████████| 1.2MB 46.6MB/s 
     |████████████████████████████████| 3.2MB 54.6MB/s 
     |████████████████████████████████| 890kB 54.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.3-cp37-none-any.whl size=114277 sha256=800f0e896ed18a6de417342db48095ae94200f55110739ad2cf03ba430c4ad9d
  Stored in directory: /root/.cache/pip/wheels/c8/0d/fa/4e28cf045da4781344e7972befb2fdf306051b225bfc290187
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=79cae87a58272c9c5dba8f8312b6ca704ebf7227dd783bd5f05bcbdb738baf16
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [9]:
import base64
import csv
import glob
import json
import numpy as np
import pandas as pd
import re
import semanticscholar as sch
import torch
from IPython.display import display, Latex, HTML
from sentence_transformers import SentenceTransformer, models
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModel

In [10]:
df_covid.drop_duplicates(['abstract'], inplace=True)
df_covid.dropna(subset=['body_text'], inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
covid_terms =['covid', 'coronavirus disease 19', 'sars cov 2', '2019 ncov', '2019ncov', '2019 n cov', '2019n cov',
              'ncov 2019', 'n cov 2019', 'coronavirus 2019', 'wuhan pneumonia', 'wuhan virus', 'wuhan coronavirus',
              'coronavirus 2', 'covid-19', 'SARS-CoV-2', '2019-nCov']
covid_terms = [elem.lower() for elem in covid_terms]
covid_terms = re.compile('|'.join(covid_terms))


In [12]:
def checkCovid(row, covid_terms):
    return bool(covid_terms.search(row['body_text'].lower()))

In [13]:
df_covid['is_covid'] = df_covid.apply(checkCovid, axis=1, covid_terms=covid_terms)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
 df_covid_only = df_covid[df_covid['is_covid']==True]
 df_covid_only = df_covid_only.reset_index(drop=True)
 len(df_covid_only)

6082

In [15]:
def preprocessing(text):
    # remove mail
    text = re.sub(r'[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}', 'MAIL', text)
    # remove doi
    text = re.sub(r'https\:\/\/doi\.org[^\s]+', 'DOI', text)
    # remove https
    text = re.sub(r'(\()?\s?http(s)?\:\/\/[^\)]+(\))?', '\g<1>LINK\g<3>', text)
    # remove single characters repeated at least 3 times for spacing error (e.g. s u m m a r y)
    text = re.sub(r'(\w\s+){3,}', ' ', text)
    # replace tags (e.g. [3] [4] [5]) with whitespace
    text = re.sub(r'(\[\d+\]\,?\s?){3,}(\.|\,)?', ' \g<2>', text)
    # replace tags (e.g. [3, 4, 5]) with whitespace
    text = re.sub(r'\[[\d\,\s]+\]', ' ',text)
     # replace tags (e.g. (NUM1) repeated at least 3 times with whitespace
    text = re.sub(r'(\(\d+\)\s){3,}', ' ',text)
    # replace '1.3' with '1,3' (we need it for split later)
    text = re.sub(r'(\d+)\.(\d+)', '\g<1>,\g<2>', text)
    # remove all full stops as abbreviations (e.g. i.e. cit. and so on)
    text = re.sub(r'\.(\s)?([^A-Z\s])', ' \g<1>\g<2>', text)
    # correctly spacing the tokens
    text = re.sub(r' {2,}', ' ', text)
    text = re.sub(r'\.{2,}', '.', text)
    # return lowercase text
    return text.lower()

In [16]:
df_covid_only['preproc_body_text'] = df_covid_only['body_text'].apply(preprocessing)

In [17]:
tokenizer = AutoTokenizer.from_pretrained("ktrapeznikov/albert-xlarge-v2-squad-v2")

In [18]:
def checkAnyStop(token_list, token_stops):
    return any([stop in token_list for stop in token_stops])

def firstFullStopIdx(token_list, token_stops):
    """
    Returns the index of first full-stop token appearing.  
    """
    idxs = []
    for stop in token_stops:
        if stop in token_list:
            idxs.append(token_list.index(stop))
    minIdx = min(idxs) if idxs else None
    return minIdx

puncts = ['!', '.', '?', ';']
puncts_tokens = [tokenizer.tokenize(x)[0] for x in puncts]

def splitTokens(tokens, punct_tokens, split_length):
    """
    To avoid splitting a sentence and lose the semantic meaning of it, a paper is splitted 
    into chunks in such a way that each chunk ends with a full-stop token (['.' ';' '?' or '!']) 
    """
    splitted_tokens = []
    while len(tokens) > 0:
        if len(tokens) < split_length or not checkAnyStop(tokens, punct_tokens):
            splitted_tokens.append(tokens)
            break
        # to not have too long parapraphs, the nearest fullstop is searched both in the previous 
        # and the next strings.
        prev_stop_idx = firstFullStopIdx(tokens[:split_length][::-1], puncts_tokens)
        next_stop_idx = firstFullStopIdx(tokens[split_length:], puncts_tokens)
        if pd.isna(next_stop_idx):
            splitted_tokens.append(tokens[:split_length - prev_stop_idx])
            tokens = tokens[split_length - prev_stop_idx:]
        elif pd.isna(prev_stop_idx):
            splitted_tokens.append(tokens[:split_length + next_stop_idx + 1])
            tokens = tokens[split_length + next_stop_idx + 1:] 
        elif prev_stop_idx < next_stop_idx:
            splitted_tokens.append(tokens[:split_length - prev_stop_idx])
            tokens = tokens[split_length - prev_stop_idx:]
        else:
            splitted_tokens.append(tokens[:split_length + next_stop_idx + 1])
            tokens = tokens[split_length + next_stop_idx + 1:] 
    return splitted_tokens

def splitParagraph(text, split_length=90):
    tokens = tokenizer.tokenize(text)
    splitted_tokens = splitTokens(tokens, puncts_tokens, split_length)
    return [tokenizer.convert_tokens_to_string(x) for x in splitted_tokens]

In [19]:
df_covid_only['body_text_parags'] = df_covid_only['preproc_body_text'].apply(splitParagraph)
df_covid_only.head()

Token indices sequence length is longer than the specified maximum sequence length for this model (3666 > 512). Running this sequence through the model will result in indexing errors


,paper_id,abstract,body_text,abstract_word_count,body_word_count,body_unique_words,is_covid,preproc_body_text,body_text_parags
0,00006903b396d50cc0037fed39916d57d50ee801,urban green space regarded contributing citize...,advantages urban green space public health urb...,191,4995,1541,True,advantages urban green space public health urb...,[advantages urban green space public health ur...
1,0000fcce604204b1b9d876dc073eb529eb5ce305,contribución de los autores rocíof menéndez co...,se calcula que en españa existen cerca de 367 ...,401,4965,1660,True,se calcula que en españa existen cerca de 367 ...,[se calcula que en espana existen cerca de 367...
2,000122a9a774ec76fa35ec0c0f6734e7e8d0c541,introduction objectives covid 19 outbreak uncl...,introduction objectives covid 19 outbreak uncl...,286,3482,1296,True,introduction objectives covid 19 outbreak uncl...,[introduction objectives covid 19 outbreak unc...
3,00031cc15aafa30b187ed2cd3790e970e5046895,background sars cov 2 infection because renal ...,increased serum creatinine variable degrees pr...,98,4190,1493,True,increased serum creatinine variable degrees pr...,[increased serum creatinine variable degrees p...
4,00033d5a12240a8684cfe943954132b43434cf48,background amplification viral ribonucleic aci...,severe acute respiratory syndrome coronavirus ...,224,1884,837,True,severe acute respiratory syndrome coronavirus ...,[severe acute respiratory syndrome coronavirus...


In [20]:
text = df_covid_only['body_text_parags'].to_frame()
body_texts = text.stack().tolist()

In [21]:
encoding_model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

In [22]:
covid_encoded = []
for body in tqdm(body_texts):
    covid_encoded.append(encoding_model.encode(body, show_progress_bar=False))

100%|██████████| 6082/6082 [11:55<00:00,  8.51it/s]


In [23]:
def computeMaxCosine(encoded_query, encodings):
    cosines = cosine_similarity(encoded_query[0].reshape(1, -1), encodings)
    return float(np.ndarray.max(cosines, axis=1))

In [24]:
def extractPapersIndexes(query, num_papers=5):
    encoded_query = encoding_model.encode([query.replace('?', '')], show_progress_bar=False)
    cosines_max = []
    for idx in range(len(covid_encoded)):
        paper = np.array(covid_encoded[idx])
        result = computeMaxCosine(encoded_query, paper)
        cosines_max.append(result)
        
    indexes_max_papers = np.array(cosines_max).argsort()[-num_papers:][::-1]
    return indexes_max_papers

In [25]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

BERT_SQUAD = "bert-large-uncased-whole-word-masking-finetuned-squad"
qa_tokenizer = AutoTokenizer.from_pretrained(BERT_SQUAD,return_dict=False)
qa_model = AutoModelForQuestionAnswering.from_pretrained(BERT_SQUAD,return_dict=False)

qa_model = qa_model.to(torch_device)
qa_model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12,

In [50]:
def answerQuestion(question, paper):
    """
    This funtion provides the best answer found by the Q&A model, the chunk containing it
    among all chunks of the input paper and the score obtained by the answer
    """
    inputs = [qa_tokenizer.encode_plus(
        question, paragraph, add_special_tokens=True, return_tensors="pt") for paragraph in paper]
    answers = []
    confidence_scores = []
    for n, Input in enumerate(inputs):
        input_ids = Input['input_ids'].to(torch_device)
        token_type_ids = Input['token_type_ids'].to(torch_device)
        if len(input_ids[0]) > 510:
            input_ids = input_ids[:, :510]
            token_type_ids =token_type_ids[:, :510]
        text_tokens = qa_tokenizer.convert_ids_to_tokens(input_ids[0])
        start_scores, end_scores = qa_model(input_ids, token_type_ids=token_type_ids)
        answer_start = torch.argmax(start_scores)
        answer_end = torch.argmax(end_scores)
        # if the start token of the answer is contained in the question, the start token is moved to
        # the first one of the chunk 
        check = text_tokens.index("[SEP]")
        if int(answer_start) <= check:
            answer_start = check+1
        answer = qa_tokenizer.convert_tokens_to_string(text_tokens[answer_start:(answer_end+1)])
        answer = answer.replace('[SEP]', '')
        confidence = start_scores[0][answer_start] + end_scores[0][answer_end]
        if answer.startswith('. ') or answer.startswith(', '):
            answer = answer[2:]
        answers.append(answer)
        confidence_scores.append(float(confidence))
    
    maxIdx = np.argmax(confidence_scores)
    confidence = confidence_scores[maxIdx]
    best_answer = answers[maxIdx]
    best_paragraph = paper[maxIdx]

    return best_answer, confidence, best_paragraph

In [51]:
def findStartEndIndexSubstring(context, answer):   
    """
    Search of the answer inside the paragraph. It returns the start and end index.
    """
    search_re = re.search(re.escape(answer.lower()), context.lower())
    if search_re:
        return search_re.start(), search_re.end()
    else:
        return 0, len(context)

In [52]:
def postprocessing(text):
    # capitalize the text
    text = text.capitalize()
    # '2 , 3' -> '2,3'
    text = re.sub(r'(\d) \, (\d)', "\g<1>,\g<2>", text)
    # full stop
    text = re.sub(r' (\.|\!|\?) (\w)', lambda pat: pat.groups()[0]+" "+pat.groups()[1].upper(), text)
    # full stop at the end
    text = re.sub(r' (\.|\!|\?)$', "\g<1>", text)
    # comma
    text = re.sub(r' (\,|\;|\:) (\w)', "\g<1> \g<2>", text)
    # - | \ / @ and _ (e.g. 2 - 3  -> 2-3)
    text = re.sub(r'(\w) (\-|\||\/|\\|\@\_) (\w)', "\g<1>\g<2>\g<3>", text)
    # parenthesis
    text = re.sub(r'(\(|\[|\{) ([^\(\)]+) (\)|\]|\})', "\g<1>\g<2>\g<3>", text)
    # "" e.g. " word "  -> "word"
    text = re.sub(r'(\") ([^\"]+) (\")', "\g<1>\g<2>\g<3>", text)
    # apostrophe  e.g. l ' a  -> l'a
    text = re.sub(r'(\w) (\') (\w)', "\g<1>\g<2>\g<3>", text)
    # '3 %' ->  '3%'
    text = re.sub(r'(\d) \%', "\g<1>%", text)
    # '# word'  -> '#word'
    text = re.sub(r'\# (\w)', "#\g<1>", text)
    # https and doi
    text = re.sub(r'(https|doi) : ', "\g<1>:", text)
    return text

In [53]:
def getSingleLOEScore(paper_idx, loe_idx):
    keywords_analysed = re.compile("|".join(keywords_list[loe_idx]))
    return len(re.findall(keywords_analysed, df_covid_only['body_text'][paper_idx]))

In [54]:
def getPaperLOE(paper_idx):
    title_search = re.compile("systematic review|meta-analysis")
    if title_search.search(df_covid_only['title'][paper_idx]):
        return "systematic review and meta-analysis"
    scores = [getSingleLOEScore(paper_idx, idx) for idx in range(len(idx_design_map))]
    sorted_indexes = np.argsort(scores)[::-1]
    for sorted_index in sorted_indexes:
        if scores[sorted_index] >= minimum_occurrencies[sorted_index]:
            return idx_design_map[sorted_index]
    return "Information not available"

In [55]:
def getAPIInformations(paper_id):
    paper_api = sch.paper(paper_id)
    if paper_api:
        return paper_api['influentialCitationCount'], paper_api['venue']
    else:
        return "Information not available", None

def getSingleContext(context, start, end):
    before_answer = context[:start]
    answer = "***" + context[start:end] + "***"
    after_answer = context[end:]
    content = before_answer + answer + after_answer
    context_answer = """<div class="single_answer">{}</div>""".format(postprocessing(content))
    return content

def getAllContexts(question, indexes_papers):
    answers_list = []
    for paper_index in indexes_papers:
        answer, conf, paragraph = answerQuestion(question, df_covid_only['body_text_parags'][paper_index])
        if answer:
            #author = df_covid_only['authors'][paper_index] if not pd.isna(df_covid_only['authors'][paper_index]) else "not available"
            #journal = df_covid_only['journal'][paper_index] if not pd.isna(df_covid_only['journal'][paper_index]) else "not available"
            #title = df_covid_only['title'][paper_index] if not pd.isna(df_covid_only['title'][paper_index]) else "not available"
            #publish_time = df_covid_only['publish_time'][paper_index] if not pd.isna(df_covid_only['publish_time'][paper_index]) else "not available"
            start, end = findStartEndIndexSubstring(paragraph, answer)
            answer_parag = getSingleContext(paragraph, start, end)
            #paper_citations_count, journal_api = getAPIInformations(df_covid_only['paper_id'][paper_index])
            #journal = journal_api if journal_api else journal
            #journal_row = scimago_jr[scimago_jr['Title'].apply(lambda x: x.lower()) == journal.lower()]
            #journal_score = journal_row['SJR'].item() if not journal_row.empty else "Information not available"
            #loe = 100 
            #getPaperLOE(paper_index)
            paper_answer =answer_parag #{ 
                # answer_parag, 
                #"title": title,
                #"journal": journal,
                #"author": author,
                #"publish_time": publish_time,
                #"journal_score": journal_score,
                #"paper_citations_count": paper_citations_count,
                #"level_of_evidence": loe
            #}
            answers_list.append(paper_answer)  
    return answers_list

In [56]:
def getFullDictQueries(data, num_papers=1):
    final_dict = {}
    for task in data.keys():
        print(f"Processing: {task}")
        task_questions = data[task]['questions']
        dict_task_quest ={}
        dict_queries = {}
        for idx, query in enumerate(task_questions):
            print(f"Getting answers from query: {query}")
            indexes_papers = extractPapersIndexes(query, num_papers=num_papers)
            dict_queries[query] = getAllContexts(query, indexes_papers)
        dict_task_quest['queries'] = dict_queries
        dict_task_quest['task_name'] = data[task]['area']
        final_dict[task] = dict_task_quest
    return final_dict

In [57]:
list_question=["Does AHCCCS have a centralized resource for medical coding resources related to COVID-19?",
               "Are there billing codes available for COVID-19 testing outside of Centers for Disease Control and Prevention (CDC) testing laboratories?",
"Will AHCCCS issue guidance regarding prior authorization expectations related to COVID-19 testing and treatment?",
"Is there a claims modifier for services related to the COVID-19 emergency?",
"Does AHCCCS cover testing for COVID-19?",
"Is there an ICD-10 diagnostic code for COVID-19?",
"How should hospitals that are reimbursed by APR-DRG bill for inpatient services related to the new ICD-10 diagnosis code for COVID-19?",
"Is IHS required to report CPT and modifiers on UB-04 claims (pertaining to COVID-19)?",
"Does AHCCCS reimburse IHS and 638 providers for services rendered at an Alternate Care Site (ACS)?",
"Can providers bill Medicaid for personal protective equipment (PPE)?",
"Does AHCCCS have guidance about Prior Authorization (PA) and Concurrent Review procedures during the COVID-19 emergency?",
"Is AHCCCS willing to suspend medical record review audits and behavioral health chart audit review processes at this time?",
"Will AHCCCS consider relaxing the timeframe for issuing Notice of Adverse Benefit Determination (NOABD) letters for non-urgent prior authorization requests?",
"Due to the need to print securely we have been unable to deploy these staff home while complying with this AHCCCS requirement?"
"Has AHCCCS received any additional grant funding to help providers address COVID-19 emergency?",
"Where can I get information and updates about the coronavirus and COVID-19?",
"Where can providers find further information about the CARES and Families First Coronavirus Response Acts?",
"Is COVID-19 laboratory testing available outside of the ADHS State Lab?",
"Is AHCCCS covering COVID-19 antibody testing?",
"Who does ADHS notify when there is a confirmed positive case of COVID-19?",
"Does AHCCCS have a centralized resource for members who have general questions or are experiencing flu-like symptoms?",
"Who is at high risk for serious illness?",
"Should I keep doctor appointments for routine visits?",
"What is the guidance on well-child visits during the COVID-19 pandemic?",
"Should elective and non-emergency medical procedures be delayed or postponed?",
"Arizonans are understandably stressed and anxious right now. What mental health resources do you recommend?",
"What businesses are deemed â€œessentialâ€ under the Governorâ€™s Executive Order issued March 23, 2020?",
"Will members lose coverage during the COVID-19 emergency?",
"Are non-Medicaid members who are covered under state-only funding, also protected from disenrollment?",
"Is AHCCCS continuing to process Medicaid renewals during the COVID-19 emergency?",
"Should providers assist members with their renewals?",
"Will AHCCCS reinstitute Prior Quarter coverage during the declared emergency?",
"Is AHCCCS aware of any opportunities for providers to secure loans during the COVID emergency?",
"During the COVID-19 emergency will AHCCCS members be required to pay premiums?",
"How does AHCCCS plan to use the additional 6.2 % in federal matching funds appropriated for Medicaid programs under the Families First Coronavirus Response Act?",
"Does the one-time COVID-19 stimulus payment and/or the unemployment benefit increase affect AHCCCS eligibility?",
"Due to the COVID-19 emergency, CMS is granting State Medicaid programs the flexibility to waive and/or modify certain Medicaid requirements. What flexibilities has AHCCCS requested?",
"How does Governor Duceyâ€™s March 24th Executive Order freezing all evictions in Arizona related to COVID for 120 days affect persons determined Seriously Mentally Ill (SMI) or persons with behavioral health needs (GMH/SU) in AHCCCS supported residential programs, including Permanent Supportive Housing, and/or Behavioral Health Residential Facilities (BHRFs), in which members pay room and board?",
"Will cell phone text and minute limits be increased or lifted for the free or low cost cell phones provided by the federal government under the Lifeline Act?",
"Medicaid applicants need to sign the AHCCCS application form. How is a signature captured when an individual applies online or by phone and does not submit a paper application?",
"Should Health Plans be educating their members and contracted providers about COVID-19?",
"Should Health Plans and FFS providers continue routine staff visits to healthcare facilities, provider practices, and membersâ€™ homes?",
"Should Health Plans continue to go onsite to provider locations to investigate immediate quality of care concerns?",
"Should ALTCS Health Plans and Tribal ALTCS programs continue to conduct in-person case management and support coordination visits with members?",
"Are Health Plans permitted to conduct targeted outreach to members at risk to ensure they know the warning signs, understand precautions, and are prepared to take appropriate action should they become ill?",
"Is the newborn notification requirement continuing during the COVID19 emergency?",
"Will AHCCCS release rules that allow SNF-based dialysis instead of only free standing dialysis centers?",
"Will AHCCCS consider waiving or suspending certain deliverables to allow greater flexibility to MCOs considering much of our workforce is telecommuting and focused on member care?",
"Do the Health Plans need to request and review COOP documents from providers?",
"Is AHCCCS willing to halt hybrid performance measure efforts in light of COVID-19?",
"Are there any changes to timeframes for behavioral health service delivery during the COVID-19 emergency for members in DCS custody?",
"Will AHCCCS consider relaxing the mandatory 72-hour turnaround time for Behavioral Health Residential Facility (BHRF) prior authorization (PA) determinations due to high volume and possible capacity issues?",
"Will Corporate Compliance Audits be suspended during the COVID-19 emergency?"]

In [58]:
list_question=[el.replace('\xa0',' ') for el in list_question]

In [59]:
list_question

['Does AHCCCS have a centralized resource for medical coding resources related to COVID-19?',
 'Are there billing codes available for COVID-19 testing outside of Centers for Disease Control and Prevention (CDC) testing laboratories?',
 'Will AHCCCS issue guidance regarding prior authorization expectations related to COVID-19 testing and treatment?',
 'Is there a claims modifier for services related to the COVID-19 emergency?',
 'Does AHCCCS cover testing for COVID-19?',
 'Is there an ICD-10 diagnostic code for COVID-19?',
 'How should hospitals that are reimbursed by APR-DRG bill for inpatient services related to the new ICD-10 diagnosis code for COVID-19?',
 'Is IHS required to report CPT and modifiers on UB-04 claims (pertaining to COVID-19)?',
 'Does AHCCCS reimburse IHS and 638 providers for services rendered at an Alternate Care Site (ACS)?',
 'Can providers bill Medicaid for personal protective equipment (PPE)?',
 'Does AHCCCS have guidance about Prior Authorization (PA) and Conc

In [62]:
data = {"Task1":
         {'area':'Transmission, incubation, and environmental stability',
          'questions': list_question
        }}
#                         "Does the environment affect transmission?",
#                         "How long does the virus persist on surfaces?",
#                         "How long individuals are contagious?",
#                         "Is personal protective equipment effective ?",
#                         "What is known about immunity?",
#                         "What is the natural history of the virus?",
#                         "What is the range of the incubation period in humans?"
                       #]}
#         "Task2":
#           {'area':"COVID-19 risk factors",
#            'questions': ["Are co-infections risk factors?",
#                         "Are male gender individuals more at risk for COVID-19?",
#                         "Are pulmunary diseases risk factors?",
#                         "Are there any public health mitigation measures considered effective?",
#                         "Do we consider chronic kidney disease a risk factor for COVID-19?",
#                         "Do we consider chronic respiratory diseases risk factors for COVID-19?",
#                         "Do we consider drinking a potential risk factor for COVID-19?",
#                         "Do we consider respiratory system diseases a risk factor for COVID-19?",
#                         "How does chronic liver disease increases the risk for COVID-19?",
#                         "How does obesity increases the risk for COVID-19?",
#                         "How does overweight increases the risk for COVID-19?",
#                         "Is cancer a risk factor for COVID-19?",
#                         "Is cardio-cerebrovascular disease a risk factor for COVID-19?",
#                         "Is cerebrovascular disease a risk factor for COVID-19?",
#                         "Is individual's age considered a potential risk factor?",
#                         "Is smoking a risk factor ?",
#                         "What do we know about risk factors related to COPD?",
#                         "What do we know about risk factors related to Diabetes?",
#                         "What do we know about risk factors related to heart diseases?",
#                         "What do we know about risk factors related to hypertension?",
#                         "What is the basic reproductive number?",
#                         "What is the serial interval?",
#                         "What is the severity of the disease?",
#                         "Which are high-risk patient groups?",
#                         "Which are the environmental risk factors?"]},
#         "Task3":
#           {'area':"Vaccines, therapeutics, interventions, and clinical studies",
#            'questions': ["Are there any drugs proven to be effective in treating COVID-19 patients?",
#                         "What is the best method to combat the hypercoagulable state seen in COVID-19?",
#                         "What is the efficacy of novel therapeutics being tested currently?"]},
        
#         "Task5":
#           {'area':"Non-pharmaceutical interventions",
#            'questions': []},
        
#         "Task6":
#           {'area':"Ethical and social science considerations",
#            'questions': []},
        
#         "Task7":
#           {'area':"Medical care",
#            'questions': []},
        
#         "Task4":
#           {'area':"Diagnostics and surveillance",
#            'questions': ["Are there diagnosis techniques based on antibodies?",
#                         "Are there diagnosis techniques based on nucleic-acid tech?",
#                         "Are there new advances in diagnosing SARS-COV-2?",
#                         "Are there point-of-care tests being developed?",
#                         "Are there rapid bed-side tests?",
#                         "How does viral load relate to disease presentations?",
#                         "How does viral load relate to likelihood of a positive diagnostic test?",
#                         "Is there any policy or protocol for screening and testing?",
#                         "What do we know about diagnostics and coronavirus?"]},
        
#         "Task9":
#           {'area':"Information sharing and inter-sectoral collaboration",
#            'questions': []},

#         "Task5":
#           {'area':"How geography affects virality",
#            'questions': ["Are there geographic variations in the mortality rate of COVID-19?",
#                         "Are there geographic variations in the rate of COVID-19 spread?",
#                         "Is there any evidence to suggest geographic based virus mutations?"]},
        
#         "Task6":
#           {'area':"Relevant factors",
#            'questions': ["Are inter/inner travel restrictions effective?",
#                         "Are multifactorial strategies effective to prevent secondary transmission?",
#                         "How does temperature and humidity affect the transmission of 2019-nCoV?",
#                         "Is case isolation effective?",
#                         "Is community contact reduction effective?",
#                         "Is personal protective equipment effective?",
#                         "Is school distancing effective?",
#                         "Is the transmission seasonal?",
#                         "Is workplace distancing effective?",
#                         "Significant changes in transmissibility in changing seasons?"]},
        
#         "Task7":
#           {'area':"Models and open questions",
#            'questions': ["Are there changes in COVID-19 as the virus evolves?",
#                         "Are there studies about phenotypic change?",
#                         "Are there studies to monitor potential adaptations?",
#                         "What do models for transmission predict?",
#                         "What is known about mutations of the virus?",
#                         "What is the human immune response to COVID-19?",
#                         "What regional genetic variations (mutations) exist?"]},
        
#         "Task8":
#           {'area':"Patient descriptions",
#            'questions': ["Can asymptomatic transmission occur during incubation?",
#                         "How many pediatric patients were asymptomatic?",
#                         "Is COVID-19 associated with cardiomyopathy and cardiac arrest?",
#                         "What do we know about disease models?",
#                         "What is the incubation period across different age groups?",
#                         "What is the Incubation Period of the Virus?",
#                         "What is the length of viral shedding after illness onset?",
#                         "What is the longest duration of viral shedding?",
#                         "What is the median viral shedding duration?",
#                         "What is the natural history of the virus from an infected person?",
#                         "Which is the proportion of patients who were asymptomatic?"]},
        
#         "Task9":
#           {'area':"Population studies",
#            'questions': ["how to communicate with health care workers?",
#                         "how to interact with high-risk elderly people?",
#                         "what is the best management of patients who are underhoused or otherwise lower socioeconomic status?",
#                         "best modes of communicating with target high-risk populations?",
#                         "What are recommendations for combating and overcoming resource failures?",
#                         "What are ways to create hospital infrastructure to prevent nosocomial outbreaks and protect uninfected patients?"]},
        
#         "Task10":
#           {'area':"Material studies",
#            'questions': ["how about adhesion to hydrophilic or phobic surfaces?",
#                         "how about decontamination based on physical science?",
#                         "How does the virus persist on different materials?",
#                         "Is there susceptibility to environmental cleaning agents?",
#                         "What do we know about viral shedding in blood?",
#                         "What do we know about viral shedding in stool?",
#                         "What do we know about viral shedding in urine?",
#                         "What do we know about viral shedding nasopharynx?"]},
#         "Task11":
#           {'area':"Miscellaneous",
#            'questions': ["Is there more than one strain in circulation?",
#                         "Are there methods to control the spread in communities?",
#                         "Which efforts have been made to identify the underlying drivers of fear?",
#                         "Are there oral medications that might potentially work?",
#                         "Which are the best ways of communicating with target high-risk populations?"]},
#         "Task12":
#          {'area':'CMS',
#           'questions': ["Is CT scan a reliable tool to detect the presence of covid-19 infection?",
#                         "Which serological rapid test is shown to be the most reliable (in terms of specificity and sensitivity) to detect the presence of covid-19 infection?",
#                         "Which is the average duration of the incubation period of covid-19 virus?",
#                         "Does the duration of the incubation period of covid-19 virus depend on individual characteristics (such as age, gender, comorbidities, etc.)?",
#                         "Does the viral load affect the severity of symptoms from covid-19?",
#                         "Is there scientific evidence that flu vaccine prevents the infection from covid-19?",
#                         "Is there scientific evidence that some blood types are more prone to be infected by covid-19?",
#                         "Are tracking apps an effective tool to prevent the spread of covid-19?",
#                         "Is there scientific evidence that warm weather reduces the spread of covid-19?",
#                         "Is there scientific evidence that conjunctivitis is a symptom of covid-19?",
#                         "What is the false positive and false negative rate in Diasorin serological rapid test?",
#                         "Are the antibodies IgM an effective measure to detect the presence of covid-19?",
#                         "What is the average persistence of IgM antibodies in the blood, for individuals infected by covid-19?",
#                         "Is there scientific evidence that some ethnic groups are more affected by covid-19?",
#                         "Which comorbidities are responsible for more severe clinical conditions caused by covid-19?",
#                         "How many days, on average, does the intensive care treatment last, for individuals infected by covid-19?"]}

      # }

In [63]:
details=getFullDictQueries(data)

Processing: Task1
Getting answers from query: Does AHCCCS have a centralized resource for medical coding resources related to COVID-19?
Getting answers from query: Are there billing codes available for COVID-19 testing outside of Centers for Disease Control and Prevention (CDC) testing laboratories?
Getting answers from query: Will AHCCCS issue guidance regarding prior authorization expectations related to COVID-19 testing and treatment?
Getting answers from query: Is there a claims modifier for services related to the COVID-19 emergency?
Getting answers from query: Does AHCCCS cover testing for COVID-19?
Getting answers from query: Is there an ICD-10 diagnostic code for COVID-19?
Getting answers from query: How should hospitals that are reimbursed by APR-DRG bill for inpatient services related to the new ICD-10 diagnosis code for COVID-19?
Getting answers from query: Is IHS required to report CPT and modifiers on UB-04 claims (pertaining to COVID-19)?
Getting answers from query: Does 

In [64]:
details

{'Task1': {'queries': {'Are Health Plans permitted to conduct targeted outreach to members at risk to ensure they know the warning signs, understand precautions, and are prepared to take appropriate action should they become ill?': ['avian influenza 63 frequent handwashing respiratory hygiene mask wearing proper utensils hand washing touching contaminated surfaces educated individuals australia also reported higher intention wear face masks pandemic influenza events 64 participants willingness perform preventive measures covid 19 current study primarily driven feeling responsibility toward health followed desire prevent coronavirus transmission people feeling coronavirus serious perception personal infection risk perceived seriousness health related consequences linked engagement disease preventive behaviors covid 19 spread relatively rapidly direct humanto human contact fighting disease challenging required ***governments inform public risks necessary precautions*** protecting others 

In [65]:
details['Task1']['queries'][list_question[0]]

['ehr previously described natural language processing nlp informatics tools belonging cogstack ecosystem 16 drugpipeline 17 medcat 18 ***medcattrainer 19*** clinician case additional validation supplemental methods cv risk factors defined recorded clinical diagnosis hypertension diabetes mellitus self reported current past smoking status absence documented cvd pre existing established cvd defined previous record 1 following diagnoses myocardial infarction mi heart failure myocarditis pericarditis endocarditis atrial ventricular ']

In [66]:
answer_list1=[];
for i in range (0,len(list_question)):
  answer_list1.append(details['Task1']['queries'][list_question[i]])
  print(details['Task1']['queries'][list_question[i]]) 

['ehr previously described natural language processing nlp informatics tools belonging cogstack ecosystem 16 drugpipeline 17 medcat 18 ***medcattrainer 19*** clinician case additional validation supplemental methods cv risk factors defined recorded clinical diagnosis hypertension diabetes mellitus self reported current past smoking status absence documented cvd pre existing established cvd defined previous record 1 following diagnoses myocardial infarction mi heart failure myocarditis pericarditis endocarditis atrial ventricular ']
['ts provided support literature search manuscript editing authors discussed approved final version manuscript the authors completed icmje ***unified competing interest form available upon request*** corresponding author declare conflicts interest']
['ebola treatment monkeys 8 aim investigate remdesivir gs 441524 plasma concentrations two critically ill covid 19 treated intravenous remdesivir moreover compartmental concentrations evaluated bronchoalveolar as

In [67]:
answer_list1[4]

['cnki net trade yearbook single n2020010198 z z020 data permanent population every county wuhan extracted official website wuhan bureau statistics http tjj wuhan gov cn constituted data analysis rather research human beings ethical approval ***institutional boards required all covid 19 cases confirmed according guideline diagnosis treatment novel coronavirus pneumonia issued*** national health commission nhc http www gov cn zhuanti 2020 02 ']

In [68]:
answer_list1[3]

['ischemic stroke contrary discharges last 10 april excluded graph avoid demonstrating false drop cases given significant number still inpatients time study sharp decrease stroke admissions observed around peak pandemic consistent anecdotal international reports describing phenomenon vanishing strokes heart attacks perio decrease severe countries multiple public service announcements released 6 urging seek immediate care signs symptoms stroke 7 stroke experts attributed phenomenon unwillingness exposed sars cov 2 overwhelmed emergency room 6 italian researchers 4 additionally hypothesized pathophysiologic mechanism behind decreased stroke occurrence based controversial role il interleukin 6 stroke 19 experimental evidence il 6 ***elevated severe covid 19 neuroprotective effect enhances*** ']

In [69]:
mannual_answer=["Yes, the AHCCCS Medical Coding Resources webpage includes a COVID-19 Medical Coding Information section and COVID-19 Emergency Medical Coding guidance.",
"Yes. The Centers for Medicare & Medicaid Services (CMS) created a new Healthcare Common Procedure Coding System (HCPCS) code for COVID-19 tests administered outside of Centers for Disease Control and Prevention (CDC) testing laboratories. This code corresponds with a Food and Drug Administration policy that allows certain laboratories to develop their own tests. Providers should use existing HCPCS code U0001 for CDC testing laboratories and the new HCPCS code U0002 for non-CDC lab tests. In addition, the American Medical Association has added a new Current Procedural Terminology (CPT) code: 87635 infectious agent detection by nucleic acid (DNA or RNA); severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) (Coronavirus disease [COVID-19]), amplified probe technique."
"Prior authorization is not permitted for COVID-19 testing or treatment.",
"Yes, AHCCCS has designated the CR modifier to be used on all claims for services provided as a result of, or related to COVID-19. It is imperative that providers begin utilizing this modifier immediately in all appropriate instances in order for AHCCCS to identify the costs of services attributable to this emergency. All other guidance regarding use of modifiers continues to be applicable.",
"Yes, AHCCCS covers COVID-19 testing. HCPCS U0001 and U0002 have an effective date of February 4, 2020. CPT 87635 has an effective date of March 13, 2020. HCPCS U0003 and U0004 have effective dates of March 18, 2020. All codes have been entered in the AHCCCS PMMIS system.",
"Yes, The World Health Organization has developed an emergency ICD-10 code for the coronavirus: U07.1, 2019-nCoV acute respiratory disease. The U07.1 is effective in PMMIS for dates of service February 4, 2020 and thereafter.  Additionally, CDCâ€™s National Center for Health Statistics will implement a new diagnosis code into the ICD 10th Revision, Clinical Modification, effective with the next update on October 1, 2020; CDC issued interim coding guidance and guidelines for health care encounters and deaths related to COVID-19.",
"AHCCCS has been notified that the 3M software will not recognize the new ICD-10 diagnosis code for COVID-19 prior to April 1, 2020.  Until the April 1, 2020 software release, hospitals should bill for other related conditions such as: Pneumonia,Acute Bronchitis,Lower Respiratory Infection,ARDS - Acute respiratory distress syndrome.The April 1, 2020 release of 3M software will recognize the newly defined ICD-10 diagnosis code, U07.1, mapped to an existing APR-DRG code under the April 1, 2020 software release.",
"HS facilities are not required to report CPT/HCPCS and modifiers on UB-04 claim forms when billing the All Inclusive Rate (AIR).",
"Per the COVID-19 Emergency Declaration Blanket Waivers & Flexibilities for Health Care Providers document released by CMS, AHCCCS will reimburse for services provided at or through an ACS, with a retroactive effective date of March 1, 2020 through the end of the emergency declaration.  Such services shall be reimbursable, so  long as those services performed are administered to a Title XIX or Title XXI eligible AHCCCS member, and are medically necessary, cost-effective, federally and state reimbursable, provided by an AHCCCS-registered provider.",
"There are no AHCCCS billing codes to cover the cost of purchasing PPE. However, some health care plans are offering enhanced provider payments and supplying PPE to providers in need. Some provider associations are also able to offer PPE. If you are in need of PPE see FAQ #4 under Provider Enrollment & Requirements.",
"Yes, please see the AHCCCS May 1, 2020 memo. This guidance does not apply to AHCCCS Fee for Service Programs. Fee for Service guidance, in coordination with tribal stakeholders, is available in this Fee for Service memo.",
"AHCCCS temporarily suspended medical record review/behavioral health chart audit efforts for providers without quality/utilization concerns through the end of the COVID-19 emergency.  While this will continue for the vast majority of audits, AHCCCS has reinstated the System of Care Practice Review (SOCPR) process effective June 1, 2020.",
"No, AHCCCS will not be extending the timeframe for Notice of Adverse Benefit Determination (NOABD) letters for non-urgent prior authorization requests.",
"Yes, the Substance Abuse and Mental Health Services Administration (SAMHSA) awarded a $2 million grant to AHCCCS on April 16, 2020 to increase mental health services infrastructure in response to the COVID-19 national emergency. Funding will be distributed through the Tribal Regional Behavioral Health Authorities/Regional Behavioral Health Authorities  to serve individuals with co-occurring Serious Mental Illness (SMI) designations and Substance Use Disorder (SUD) service support needs. The grant requires: 70 percent to provide direct services to those with co-occurring SMI and SUD needs; 10 percent for healthcare workers with mental health needs (less severe than SMI) requiring mental health care as a result of COVID-19; and 20 percent for all other individuals with mental health needs less severe than SMI.",
"Information about COVID-19 is available from the Centers for Disease Control (CDC) and the Arizona Department of Health Services.  Additionally, the general public in Arizona can call the Arizona Poison Control System at 1-844-542-8201 or the statewide COVID-19 Hotline, available 8 a.m. to 8 p.m. daily, in English and Spanish, by dialling 2-1-1 in Arizona. The COVID-19 hotline is administered by 2-1-1 Arizona and the Crisis Response Network, and can address questions about how to prepare for and prevent COVID-19 spread; testing information for COVID-19; at-risk populations; what to do if an individual gets sick; COVID-19 and animals; and other resources for accurate, reliable, and up-to-date information.",
"CARES and FFCR Act guidance and FAQs on Medicaid.gov.",
"Yes, as of March 16, 2020 COVID-19 testing is now available through private labs as well as the state lab. However, the COVID-19 test requires a specimen to be collected by a physician or other authorized healthcare providers. Do not visit a laboratory location to request the COVID-19 test. Instead, contact your physician or other authorized healthcare provider for guidance regarding getting tested. Your healthcare provider can best advise whether you meet criteria for testing and their office's availability of testing supplies.",
"Yes, as outlined in Section 6004 of the Families First Coronavirus Response Act (FFCRA). Further information is available at the CDC and on the ADHS websites.",
"ADHS notifies the memberâ€™s ordering provider and the memberâ€™s County Public Health Department.",
"Yes. AHCCCS has created a web page to address Medicaid-related questions from providers and contractors about COVID-19 at azahcccs.gov/AHCCCS/AboutUs/covid19. ",
"According to the CDC's Situation Summary,, pandemic COVID-19 poses a serious public health risk. Per the Arizona Department of Health Services Frequently Asked.",
"Please call your health care provider regarding scheduled, routine visits.",
"Please see the most recent guidelines from the American Academy of Pediatrics.",
"Following a CMS recommendation, Governor Doug Ducey announced an Executive Order on March 19, 2020 that halts all elective surgeries in the state of Arizona in order to free up medical resources and maintain the capacity for hospitals and providers to continue offering vital services. On April 22, the Governor announced an Executive Order removing restrictions on elective surgeries. Effective May 1, hospitals, health care facilities and providers that meet certain preparedness criteria can resume elective surgeries. Arizona Department of Health Services has posted FAQs about elective surgeries and  an exemption request form.",
"AHCCCS covers a full array of behavioral health services for members, and crisis services are available to anyone in Arizona, regardless of insurance coverage. In addition, we have added behavioral health resources offered by the CDC and SAMHSA on the COVID-19 webpage.",
"To stem the spread of COVID-19, Governor Ducey issued an executive order clarifying businesses and operations that are deemed â€œessentialâ€ by the state, including, but are not limited to, the following roles and/or job functions: Healthcare and Public Health Operations: Hospitals, clinics, dental offices, pharmacies.",
"No, per the Families First Coronavirus Response Act, AHCCCS will not disenroll members during the emergency (with the exception of death, those who move out of state, or those who voluntarily withdraw).",
"No, members who are not federally funded under Title XIX (for example, members with a Serious Mental Illness determination who do not qualify for Medicaid) are not exempt from disenrollment during the public health emergency.",
"Yes, while we are not disenrolling members (with the exception of death, those who move out of state, or those who voluntarily withdraw), we are continuing to process annual renewals. Providers are encouraged to continue to assist members with their renewals.",
"Prior Quarter, or retroactive coverage, is currently available to children under age 19, pregnant women and women in a post partum period.  For individuals other than children, pregnant women and women in a post partum period, approved applications are effective  as of the first day of the month in which the application is submitted.  Individuals are encouraged to submit an application immediately if they are experiencing a health issue, become uninsured or experience a change in circumstance that may make them eligible for AHCCCS Medical Assistance. Apply for AHCCCS.",
"On March 27, 2020, the Coronavirus Aid, Relief, and Economic Security Act of 2020 (CARES) passed, including immediate loan assistance for small businesses. Providers may wish to review loan opportunities authorized under the Act to determine if they qualify for this stimulus funding. Stimulus loans are not under the purview of the Medicaid program and AHCCCS does not play a role in the administration of small business loan assistance.",
"No.  For the duration of the COVID-19 emergency, AHCCCS has suspended premium payments for the KidsCare and Freedom to Work programs as well as copayments for the Transitional Medical Assistance (TMA) and Adult populations.  Members who have already paid March 2020 premiums will be credited, and individuals who were disenrolled in March will be reinstated for the duration of the emergency.",
"We are aware of stakeholder interest in how AHCCCS plans to use the enhanced federal funding available through the Families First Coronavirus Response Act.  We continue to evaluate how to best use the funds, in light of the expected member growth due to the COVID-19 emergency.  In order to receive the match, AHCCCS is required not to disenroll members, and expects membership growth as the existing members remain enrolled and new members are added.  As such, AHCCCS must first ensure that the additional funding will cover the cost of the enrollment growth that will occur. AHCCCS is prohibited from disenrolling members through the end of the month in which the declaration ends.",
"The one-time COVID-19 stimulus payments and the Unemployment Insurance Federal Pandemic Unemployment Compensation (FPUC) benefits will not affect AHCCCS eligibility. See the Arizona Department of Economic Service PEUC website for more information on how FPUC, Unemployment Insurance, and Pandemic Unemployment Assistance payments may affect Medicaid eligibility.",
"AHCCCS has requested various flexibilities to waive and/or modify certain Medicaid requirements under the 1135 and 1115 Waivers and the State Plan Amendment. CMS has begun to review and grant approval of these requests. As approval is granted, AHCCCS is making operational decisions regarding implementation, and tracking progress in the Status of AHCCCS Emergency Authority Requests document on this page.",
"AHCCCS intends for the Executive Order to apply to all AHCCCS and/or Medicaid funded permanent supportive and transitional housing programs as well as services settings in which AHCCCS members pay for room and board such as BHRFs, assisted living facilities, and group homes. Residential stability is critical for members to comply with mandated stay at home orders. To this end, there should be no disruption of service or alteration of existing subsidies for persons residing in AHCCCS funded Permanent Supportive and Transitional Living programs for persons determined SMI. Similarly, these protections should be broadly applied to any service setting whereby AHCCCS members pay for room and board.",
"In all situations during this crisis, AHCCCS encourages increased care coordination of members with behavioral health needs in housing or residential programs/settings to ensure housing/placement security during this crisis.  This will allow members to maintain their residential setting and comply with social distancing and stay at home practices required under our State and National COVID response orders for their own health and the health of our community.",
"AHCCCS has learned that some Lifeline Act cell phone providers are lifting limits and providing unlimited minutes and texting due to the COVID-19 emergency. Information is available at: www.obamaphone.com and www.usac.org/lifeline.",
"When an individual applies using the HEAplus.gov online application, the system captures an electronic signature. When an individual applies over the phone, the state employee or community assistor captures a voice signature. During the COVID-19 public health emergency, community assistors are authorized to obtain verbal consent to act on the applicant's behalf and complete the application.",
"Yes, Health Plans and FFS Programs should be proactively educating members and providers utilizing information from the Centers for Disease Control (CDC), the Arizona Department of Health Services (ADHS), and other applicable entities outlined in the AHCCCS Contractor Operations Manual (ACOM) 404, Attachment A.  If the only source of information in member education materials is one of the entities listed in  ACOM 404, Attachment A, it does not need to be submitted to AHCCCS for review or approval.",
"We highly recommend that, when feasible, both Health Plans and FFS Programs and providers leverage technological platforms to conduct virtual visits whenever possible.  For example, a 180 day case manager visit for an Arizona Long Term Care System (ALTCS) member located at a Skilled Nursing Facility (SNF) can be done via telehealth.",
"In accordance with CMS guidance, AHCCCS is requesting that Health Plans and FFS Programs maintain investigative efforts for serious health and safety threats (e.g. allegations of abuse). Health Plans and FFS Programs should do their part to support sound infection control protocols, including limiting the number of staff that go onsite for the investigation, implementation of preventive practices such as handwashing and use of hand sanitizer before engaging with members and staff, and assessment of staff health before entering the facility.   If Health  Plans and FFS Programs encounter issues with ability to conduct onsite/in-person investigation of immediate quality of care concerns, please notify AHCCCS QM of the limitation at CQM@azahcccs.gov.",
"Per AMPM 1620-E, case managers (and support coordinators) must conduct in-person visits for members residing in a skilled nursing facility every 180 days and to a member receiving HCBS or acute-only services every 90 days.  AHCCCS is suspending the in-person requirement at this time.  Case managers should continue to reach out to members telephonically to complete monitoring/reassessment efforts to the extent possible.  If members are not able to be reached via telephone or other electronic means, outreach attempts should be documented in the memberâ€™s case management file.",
"Yes, AHCCCS encourages health plans to conduct targeted outreach to at risk members.  We highly recommend that Health Plans leverage technological platforms to conduct virtual visits whenever possible.",
"The Newborn notification requirements have NOT changed and remain in place.",
"AHCCCS is adopting CMS guidance for dialysis treatment for the duration of the COVID emergency.  CMS is waiving the requirement that dialysis facilities have to provide services directly on its main premises or on other premises that are contiguous with the main premises. Dialysis facilities may enter nursing home/skilled nursing home facilities to provide dialysis service to its patients in those settings in order to limit community exposure. CMS continues to require that services provided to these nursing home residents are under the direction of the same governing body and professional staff as the residentâ€™s usual Medicare-certified dialysis facility. Further, in order to ensure that care is safe, effective and is provided by trained and qualified personnel, CMS requires that the dialysis facility staff: furnish all dialysis care and services, provide all equipment and supplies necessary, maintain equipment and supplies in the nursing home, and complete all equipment maintenance, cleaning and disinfection using appropriate infection control procedures and manufacturerâ€™s instructions for use. Dialysis services delivered in a nursing home or skilled nursing facility should continue to be billed to Medicare as appropriate for dual eligible members.",
"Yes, AHCCCS is currently evaluating where flexibility can be offered for deliverables.  Additional guidance will be issued in the near future.  If an MCO has a specific request, please send it to your AHCCCS Operations/Compliance Officer.",
"During the MCO-AHCCCS Weekly COVID-19 call on 3/16, Director Snyder conveyed the message that AHCCCS expects the health plans to check in with critical provider types to ensure these providers, such as Mobile Crisis providers, Stabilization Crisis providers, Long Term Care facilities, and Outpatient Treatment Program (OTP) providers have Continuity of Operations Plans (COOP) and have reviewed them with staff. This expectation to check in does not mean that the health plans need to request copies of providersâ€™ COOP documents."
"After careful consideration, AHCCCS has directed HSAG to suspend all medical record procurement and abstraction activities for the CYE 2018 Hybrid Performance Measure Calculations. At this time, AHCCCS is prioritizing MCO and provider response to COVID-19. Once the COVID-19 emergency response has passed, AHCCCS will re-evaluate project timelines and determine next steps for hybrid audits.",
"No, there are no changes to behavioral health response timeframes for children in DCS custody. For example, the Rapid Response is still required within 72 hours after referral from DCS. Please refer to ACOM 417 and ACOM 449 for additional information on AHCCCS timeframe requirements for children in DCS custody.",
]




In [70]:
mannual_answer

['Yes,\xa0the\xa0AHCCCS\xa0Medical\xa0Coding\xa0Resources\xa0webpage\xa0includes\xa0a\xa0COVID-19\xa0Medical\xa0Coding\xa0Information\xa0section\xa0and\xa0COVID-19\xa0Emergency\xa0Medical\xa0Coding\xa0guidance.',
 'Yes.\xa0The\xa0Centers\xa0for\xa0Medicare\xa0&\xa0Medicaid\xa0Services\xa0(CMS)\xa0created\xa0a\xa0new\xa0Healthcare\xa0Common\xa0Procedure\xa0Coding\xa0System\xa0(HCPCS)\xa0code\xa0for\xa0COVID-19\xa0tests\xa0administered\xa0outside\xa0of\xa0Centers\xa0for\xa0Disease\xa0Control\xa0and\xa0Prevention\xa0(CDC)\xa0testing\xa0laboratories.\xa0This\xa0code\xa0corresponds\xa0with\xa0a\xa0Food\xa0and\xa0Drug\xa0Administration\xa0policy\xa0that\xa0allows\xa0certain\xa0laboratories\xa0to\xa0develop\xa0their\xa0own\xa0tests.\xa0Providers\xa0should\xa0use\xa0existing\xa0HCPCS\xa0code\xa0U0001\xa0for\xa0CDC\xa0testing\xa0laboratories\xa0and\xa0the\xa0new\xa0HCPCS\xa0code\xa0U0002\xa0for\xa0non-CDC\xa0lab\xa0tests.\xa0In\xa0addition,\xa0the\xa0American\xa0Medical\xa0Association\xa0has\xa

In [71]:
mannual_answer=[el.replace('\xa0',' ') for el in mannual_answer]

In [72]:
mannual_answer

['Yes, the AHCCCS Medical Coding Resources webpage includes a COVID-19 Medical Coding Information section and COVID-19 Emergency Medical Coding guidance.',
 'Yes. The Centers for Medicare & Medicaid Services (CMS) created a new Healthcare Common Procedure Coding System (HCPCS) code for COVID-19 tests administered outside of Centers for Disease Control and Prevention (CDC) testing laboratories. This code corresponds with a Food and Drug Administration policy that allows certain laboratories to develop their own tests. Providers should use existing HCPCS code U0001 for CDC testing laboratories and the new HCPCS code U0002 for non-CDC lab tests. In addition, the American Medical Association has added a new Current Procedural Terminology (CPT) code: 87635 infectious agent detection by nucleic acid (DNA or RNA); severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) (Coronavirus disease [COVID-19]), amplified probe technique.Prior authorization is not permitted for COVID-19 testing or

In [74]:
from itertools import chain

def get_unigram_count(tokens):
    count_dict = dict()
    for t in tokens:
        if t in count_dict:
            count_dict[t] += 1
        else:
            count_dict[t] = 1

    return count_dict


class Rouge:
    beta = 1

    @staticmethod
    def my_lcs_grid(x, y):
        n = len(x)
        m = len(y)

        table = [[0 for i in range(m + 1)] for j in range(n + 1)]

        for j in range(m + 1):
            for i in range(n + 1):
                if i == 0 or j == 0:
                    cell = (0, 'e')
                elif x[i - 1] == y[j - 1]:
                    cell = (table[i - 1][j - 1][0] + 1, '\\')
                else:
                    over = table[i - 1][j][0]
                    left = table[i][j - 1][0]

                    if left < over:
                        cell = (over, '^')
                    else:
                        cell = (left, '<')

                table[i][j] = cell

        return table

    @staticmethod
    def my_lcs(x, y, mask_x):
        table = Rouge.my_lcs_grid(x, y)
        i = len(x)
        j = len(y)

        while i > 0 and j > 0:
            move = table[i][j][1]
            if move == '\\':
                mask_x[i - 1] = 1
                i -= 1
                j -= 1
            elif move == '^':
                i -= 1
            elif move == '<':
                j -= 1

        return mask_x

    @staticmethod
    def rouge_l(cand_sents, ref_sents):
        lcs_scores = 0.0
        cand_unigrams = get_unigram_count(chain(*cand_sents))
        ref_unigrams = get_unigram_count(chain(*ref_sents))
        for cand_sent in cand_sents:
            cand_token_mask = [0 for t in cand_sent]
            cand_len = len(cand_sent)
            for ref_sent in ref_sents:
                # aligns = []
                # Rouge.lcs(ref_sent, cand_sent, aligns)
                Rouge.my_lcs(cand_sent, ref_sent, cand_token_mask)

                # for i in aligns:
                #     ref_token_mask[i] = 1
            # lcs = []
            cur_lcs_score = 0.0
            for i in range(cand_len):
                if cand_token_mask[i]:
                    token = cand_sent[i]
                    if cand_unigrams[token] > 0 and ref_unigrams[token] > 0:
                        cand_unigrams[token] -= 1
                        ref_unigrams[token] -= 1
                        cur_lcs_score += 1

                        # lcs.append(token)

            # print ' '.join(lcs)

            lcs_scores += cur_lcs_score

        # print "lcs_scores: %d" % lcs_scores
        ref_words_count = sum(len(s) for s in ref_sents)
        # print "ref_words_count: %d" % ref_words_count
        cand_words_count = sum(len(s) for s in cand_sents)
        # print "cand_words_count: %d" % cand_words_count

        precision = lcs_scores / cand_words_count
        recall = lcs_scores / ref_words_count
        f_score = (1 + Rouge.beta ** 2) * precision * recall / (recall +
                                                                Rouge.beta ** 2 * precision + 1e-7) + 1e-6  # prevent underflow
        return precision, recall, f_score

In [112]:
import nltk
nltk.download('punkt')

from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu



def get_bleu_score(reference, candidate):
    print(word_tokenize(reference))
    print(word_tokenize(candidate))
    score = sentence_bleu(word_tokenize(reference), word_tokenize(candidate), weights=[0.5, 0.5])
    return '%f' % score

def get_rouge_score(reference_summary, generated_summary):
    r = Rouge()
    [precision, recall, f_score] = r.rouge_l([generated_summary], [reference_summary])
    print("Precision is :" + str(precision) + "\nRecall is :" + str(recall) + "\nF Score is :" + str(f_score))
    return (precision), (recall), (f_score)


avg_1=0
avg_v1=0
avg_v2=0
avg_v3=0
count1=0
v1=0
v2=0
v3=0
for i in range(0,len(mannual_answer)):
  try:
    system_generated_summary = answer_list1[i][0]
    manual_summmary = mannual_answer[i]
    print((float(get_bleu_score(system_generated_summary, manual_summmary))))
    print(get_rouge_score(system_generated_summary, manual_summmary))
    avg_1=avg_1+(float(get_bleu_score(system_generated_summary, manual_summmary)))
    count1=count1+1
    v1,v2,v3=((get_rouge_score(system_generated_summary, manual_summmary)))
    avg_v1=avg_v1+float(v1)
    avg_v2=avg_v2+float(v2)
    avg_v3=avg_v3+float(v3)

  except:
    print("Not Compare")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['ehr', 'previously', 'described', 'natural', 'language', 'processing', 'nlp', 'informatics', 'tools', 'belonging', 'cogstack', 'ecosystem', '16', 'drugpipeline', '17', 'medcat', '18', '***medcattrainer', '19***', 'clinician', 'case', 'additional', 'validation', 'supplemental', 'methods', 'cv', 'risk', 'factors', 'defined', 'recorded', 'clinical', 'diagnosis', 'hypertension', 'diabetes', 'mellitus', 'self', 'reported', 'current', 'past', 'smoking', 'status', 'absence', 'documented', 'cvd', 'pre', 'existing', 'established', 'cvd', 'defined', 'previous', 'record', '1', 'following', 'diagnoses', 'myocardial', 'infarction', 'mi', 'heart', 'failure', 'myocarditis', 'pericarditis', 'endocarditis', 'atrial', 'ventricular']
['Yes', ',', 'the', 'AHCCCS', 'Medical', 'Coding', 'Resources', 'webpage', 'includes', 'a', 'COVID-19', 'Medical', 'Coding', 'Information', 'section', 'and', 'COVI

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Precision is :0.18874560375146543
Recall is :0.6571428571428571
F Score is :0.29326143891941714
['ebola', 'treatment', 'monkeys', '8', 'aim', 'investigate', 'remdesivir', 'gs', '441524', 'plasma', 'concentrations', 'two', 'critically', 'ill', 'covid', '19', 'treated', 'intravenous', 'remdesivir', 'moreover', 'compartmental', 'concentrations', 'evaluated', 'bronchoalveolar', 'aspirate', 'bas', 'csf', 'sample', 'them', 'the', 'conducted', 'accordance', 'declaration', '***helsinki', 'national', 'institutional', 'standards***', 'informed', 'consent', 'obtained', 'family', 'members', 'data', 'collected', 'anonymously', 'the', 'two', 'female', 'patient', '1', 'pt1', 'male', 'patient', '2', 'pt2', '66', '67', 'years', 'old', 'respectively', 'residents', 'city', 'wuhan', 'china', 'travelled', 'italy', 'in', 'january', 'developed', 'fever', 'respiratory', 'symptoms', 'subsequently', 'admitted', 'national', 'institute', 'infectious', 'diseases', 'lazzaro', 'spallanzani', 'rome', 'italy', 'nasoph

In [88]:
avg_1=avg_1/count1

In [89]:
avg_1

0.07935367441860468

In [113]:
avg_v1=avg_v1/count1

In [114]:
avg_v1

0.49622066631787715

In [115]:
avg_v2=avg_v2/count1

In [116]:
avg_v2

0.3095507688191365

In [117]:
avg_v3=avg_v3/count1

In [118]:
avg_v3

0.3346630683780721

In [119]:
import pickle
#actual model is MODEL_1
name = '/content/drive/MyDrive/pickle1_baseline-1'
pickle_file = open(name, "wb")       
pickle.dump(qa_model,pickle_file)
pickle.dump(covid_encoded,pickle_file)
pickle_file.close()
#code to dump the model